In [23]:
#libraries
from palmerpenguins import load_penguins
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.pipeline import Pipeline
from sklearn.compose import make_column_selector, ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, PolynomialFeatures
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet, LogisticRegression 
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import r2_score
from sklearn.model_selection import GridSearchCV

penguins = load_penguins()
penguins.head()

#remove NA's from penguins dataset
penguins = penguins.dropna()

In [24]:
#want pipeline to predict genfoo species

#make gentoo column for penguins dataset

#true/falses is okay for modeling, multiply by 1 to make integers
penguins['is_gentoo'] = (penguins['species'] == 'Gentoo') * 1


Use the other variables in the dataset and a logistic regression model to classify penguins as either “Gentoo” or “Not”. What is your training accuracy?

In [33]:
#pipeline


#Create a pipeline that includes all the columns as predictors for Salary, and performs ordinary linear regression

#X = penguins.drop(["is_gentoo", "species"], axis = 1)
X = penguins[["bill_depth_mm", "sex"]]
y = penguins["is_gentoo"]


ct = ColumnTransformer(
  [
    ("dummify", 
    #handle_unknown: ignore observations for unknown values in category variable
    OneHotEncoder(sparse_output = False, handle_unknown='ignore'),
    make_column_selector(dtype_include=object)),
    ("standardize", 
    StandardScaler(), 
    make_column_selector(dtype_include=np.number))
  ],
  remainder = "passthrough"
)

lr_pipeline_logistic = Pipeline(
  [("preprocessing", ct),
  ("logistic_regression", LogisticRegression())]
).set_output(transform="pandas")

In [37]:
#Use cross-validation to estimate the MSE you would expect if you used this pipeline to predict 1989 salaries.

#score how well the model did
#R^2
scores = cross_val_score(lr_pipeline_logistic, X, y, cv=5, scoring='r2')
r2 = scores.mean()

#MSE
scores = cross_val_score(lr_pipeline_logistic, X, y, cv=5, scoring='accuracy')
accuracy = scores.mean()

print("r2: ", r2, " accuracy: ", accuracy)

r2:  0.9336282440175261  accuracy:  0.9848937132519222
r2:  0.9336282440175261  accuracy:  0.9848937132519222


Let's use the other variables to predict species. Prepare your data and fit the following models on the entire dataset:

- Two kNN models (for different values of K)
- Two decision tree models (for different complexities of trees)

Compute the following, for each of your models, on test data. Keep in mind that you may need to stratify your creation of the training and test data.

Confusion matrix
- Overall Accuracy
- Precision, Recall, AUC, and F1-score for each species
- Create one ROC plot for the species of your choice.